# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks, code quality and readability (no single-letter variables, PEP8 compliant) etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [1]:
import findspark
findspark.init()

import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, LongType, ShortType

sc = SparkContext(appName="ClickstreamAnalysis")
spark = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-10-08 17:30:36,737 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [2]:
schema = StructType([
    StructField("user_id", ShortType(), True),
    StructField("session_id", ShortType(), True),
    StructField("event_type", StringType(), True),
    StructField("event_page", StringType(), True),
    StructField("timestamp", LongType(), True)
])

df = spark.read.csv("hdfs:/data/clickstream.csv", header=True, sep="\t", schema=schema)
df.show(5)
df.createOrReplaceTempView("df")

+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1695584127|
|    562|       507|       event|      main|1695584134|
|    562|       507|       event|      main|1695584144|
|    562|       507|       event|      main|1695584147|
|    562|       507|wNaxLlerrorU|      main|1695584154|
+-------+----------+------------+----------+----------+
only showing top 5 rows



## 1 Using Spark RDD

In [3]:
from operator import add


def pretty_print(top_routes):
    print("+--------------------+-----+")
    print("|               route|count|")
    print("+--------------------+-----+")
    
    for route, count in top_routes:
        truncated_route = (route[:17] + '...') if len(route) > 20 else route.ljust(20, ' ')
        aligned_count = str(count).rjust(5, ' ')
        print(f"|{truncated_route}|{aligned_count}|")
    
    print("+--------------------+-----+")

In [23]:
df_rdd = df.rdd

In [24]:
def map_func(row):
    try:
        error_flag = 1 if 'error' in row.event_type else 0
        return [((row.user_id, row.session_id), [(row.timestamp, error_flag, row.event_page)])]
    except Exception as e:
        print(f"Error in map_func: {str(e)}, with row: {row}")
        return []

In [25]:
mapped_rdd = df_rdd.flatMap(map_func)
mapped_rdd.take(5)

[((562, 507), [(1695584127, 0, 'main')]),
 ((562, 507), [(1695584134, 0, 'main')]),
 ((562, 507), [(1695584144, 0, 'main')]),
 ((562, 507), [(1695584147, 0, 'main')]),
 ((562, 507), [(1695584154, 1, 'main')])]

In [29]:
grouped_sorted_rdd = mapped_rdd.reduceByKey(lambda a,b: a+b).mapValues(lambda v: sorted(v, key=lambda x: x[0]))
grouped_sorted_rdd.take(1)

[((1974, 700),
  [(1696091444, 0, 'main'),
   (1696091654, 0, 'main'),
   (1696091785, 0, 'online'),
   (1696092968, 0, 'main'),
   (1696093073, 0, 'main'),
   (1696093508, 0, 'online'),
   (1696093548, 0, 'rabota'),
   (1696093620, 0, 'bonus'),
   (1696093655, 0, 'bonus'),
   (1696093851, 0, 'bonus'),
   (1696093918, 0, 'archive'),
   (1696093967, 0, 'archive'),
   (1696094104, 0, 'tariffs'),
   (1696094159, 0, 'bonus'),
   (1696094341, 0, 'bonus'),
   (1696094925, 0, 'vklad'),
   (1696095152, 0, 'vklad'),
   (1696095430, 0, 'vklad'),
   (1696096365, 0, 'news'),
   (1696096405, 0, 'news'),
   (1696267341, 0, 'main'),
   (1696267408, 0, 'main'),
   (1696267991, 0, 'bonus'),
   (1696268484, 0, 'online'),
   (1696268499, 0, 'rabota'),
   (1696269064, 0, 'rabota'),
   (1696270284, 0, 'rabota'),
   (1696270671, 0, 'archive'),
   (1696270827, 0, 'archive'),
   (1696271112, 0, 'archive'),
   (1696271830, 0, 'archive'),
   (1696271959, 0, 'tariffs'),
   (1696272019, 0, 'tariffs'),
   (1696272

In [27]:
def cumsum_errors(events_list):
    cumsum = 0
    new_list = []
    for event in events_list:
        timestamp, error_flag, event_page = event
        cumsum += error_flag
        new_list.append((timestamp, cumsum, event_page))
    return new_list

cumsum_rdd = grouped_sorted_rdd.mapValues(cumsum_errors)
cumsum_rdd.take(1)

[((2209, 541),
  [(1695585442, 0, 'main'),
   (1695585480, 0, 'main'),
   (1695585553, 0, 'bonus'),
   (1695585571, 0, 'bonus'),
   (1695585593, 0, 'online'),
   (1695585610, 0, 'online'),
   (1695585733, 0, 'online'),
   (1695585783, 0, 'online'),
   (1695585792, 0, 'internet'),
   (1695585812, 0, 'internet'),
   (1695585822, 0, 'internet'),
   (1695585842, 0, 'internet'),
   (1695585878, 0, 'news'),
   (1695585918, 0, 'news'),
   (1695585934, 0, 'news'),
   (1695585951, 0, 'news'),
   (1695585994, 0, 'main'),
   (1695586015, 0, 'main')])]

In [15]:
def contains_error(events_list):
    for _, error_flag, _ in events_list:
        if error_flag == 1:
            return True
    return False


no_error_rdd = cumsum_rdd.filter(lambda x: not contains_error(x[1]))
no_error_rdd.take(1)

[((1974, 700),
  [(1696270671, 0, 'archive'),
   (1696270827, 0, 'archive'),
   (1696271112, 0, 'archive'),
   (1696271830, 0, 'archive'),
   (1696271959, 0, 'tariffs'),
   (1696272019, 0, 'tariffs'),
   (1696272428, 0, 'tariffs'),
   (1696272595, 0, 'tariffs'),
   (1696272766, 0, 'bonus'),
   (1696273019, 0, 'bonus'),
   (1696273021, 0, 'bonus'),
   (1696274035, 0, 'bonus'),
   (1696274108, 0, 'bonus'),
   (1696274751, 0, 'rabota'),
   (1696275058, 0, 'rabota'),
   (1696276067, 0, 'internet'),
   (1696277135, 0, 'internet'),
   (1696277499, 0, 'vklad'),
   (1696277896, 0, 'archive'),
   (1696278333, 0, 'archive'),
   (1696278697, 0, 'archive'),
   (1696279104, 0, 'archive'),
   (1696280120, 0, 'rabota'),
   (1696091444, 0, 'main'),
   (1696091654, 0, 'main'),
   (1696091785, 0, 'online'),
   (1696092968, 0, 'main'),
   (1696093073, 0, 'main'),
   (1696093508, 0, 'online'),
   (1696093548, 0, 'rabota'),
   (1696093620, 0, 'bonus'),
   (1696093655, 0, 'bonus'),
   (1696093851, 0, 'bonus

In [6]:
def reduce_func(x, y):
    try:
        return x + y
    except Exception as e:
        print(f"Error in reduce_func: {str(e)}, with x: {x}, y: {y}")
        return []
    

cumsum_rdd = mapped_rdd.reduceByKey(reduce_func)
cumsum_rdd.take(1)

[((1974, 700),
  [(1696270671, 0, 'archive'),
   (1696270827, 0, 'archive'),
   (1696271112, 0, 'archive'),
   (1696271830, 0, 'archive'),
   (1696271959, 0, 'tariffs'),
   (1696272019, 0, 'tariffs'),
   (1696272428, 0, 'tariffs'),
   (1696272595, 0, 'tariffs'),
   (1696272766, 0, 'bonus'),
   (1696273019, 0, 'bonus'),
   (1696273021, 0, 'bonus'),
   (1696274035, 0, 'bonus'),
   (1696274108, 0, 'bonus'),
   (1696274751, 0, 'rabota'),
   (1696275058, 0, 'rabota'),
   (1696276067, 0, 'internet'),
   (1696277135, 0, 'internet'),
   (1696277499, 0, 'vklad'),
   (1696277896, 0, 'archive'),
   (1696278333, 0, 'archive'),
   (1696278697, 0, 'archive'),
   (1696279104, 0, 'archive'),
   (1696280120, 0, 'rabota'),
   (1696091444, 0, 'main'),
   (1696091654, 0, 'main'),
   (1696091785, 0, 'online'),
   (1696092968, 0, 'main'),
   (1696093073, 0, 'main'),
   (1696093508, 0, 'online'),
   (1696093548, 0, 'rabota'),
   (1696093620, 0, 'bonus'),
   (1696093655, 0, 'bonus'),
   (1696093851, 0, 'bonus

In [7]:
def contains_error(events_list):
    for _, error_flag, _ in events_list:
        if error_flag == 1:
            return True
    return False


no_error_rdd = cumsum_rdd.filter(lambda x: not contains_error(x[1]))
no_error_rdd.take(1)

[((2209, 541),
  [(1695585442, 0, 'main'),
   (1695585480, 0, 'main'),
   (1695585553, 0, 'bonus'),
   (1695585571, 0, 'bonus'),
   (1695585593, 0, 'online'),
   (1695585610, 0, 'online'),
   (1695585733, 0, 'online'),
   (1695585783, 0, 'online'),
   (1695585792, 0, 'internet'),
   (1695585812, 0, 'internet'),
   (1695585822, 0, 'internet'),
   (1695585842, 0, 'internet'),
   (1695585878, 0, 'news'),
   (1695585918, 0, 'news'),
   (1695585934, 0, 'news'),
   (1695585951, 0, 'news'),
   (1695585994, 0, 'main'),
   (1695586015, 0, 'main')])]

In [273]:
route_counts_rdd = routes_rdd.reduceByKey(lambda x, y: x + y)
route_counts_rdd.take(1)

[('main-tariffs-rabota-news', 18)]

In [276]:
top_routes_rdd = route_counts_rdd.takeOrdered(30, key=lambda x: -x[1])

In [277]:
pretty_print(top_routes_rdd)

+--------------------+-----+
|               route|count|
+--------------------+-----+
|main                | 5778|
|main-archive        |  845|
|main-rabota         |  799|
|main-internet       |  694|
|main-bonus          |  643|
|main-news           |  597|
|main-tariffs        |  510|
|main-online         |  439|
|main-vklad          |  387|
|main-archive-rabota |  156|
|main-rabota-archive |  153|
|main-bonus-rabota   |  134|
|main-internet-rabota|  130|
|main-news-rabota    |  127|
|main-internet-arc...|  127|
|main-archive-news   |  124|
|main-rabota-internet|  122|
|main-archive-inte...|  122|
|main-rabota-news    |  121|
|main-archive-bonus  |  119|
|main-rabota-bonus   |  119|
|main-bonus-archive  |  119|
|main-news-archive   |  109|
|main-internet-bonus |  103|
|main-internet-news  |   99|
|main-archive-online |   98|
|main-archive-tariffs|   97|
|main-tariffs-inte...|   93|
|main-tariffs-archive|   90|
|main-news-internet  |   90|
+--------------------+-----+


## 2 Using Spark SQL

In [226]:
spark.sql(
"""
WITH
    EXTRACT_ERRORS
AS
(
    SELECT
        *,
        SUM(CASE WHEN event_type LIKE '%error%' THEN 1 ELSE 0 END) 
            OVER window_spec AS cumsum_error
    FROM df
    WINDOW window_spec AS (
        PARTITION BY user_id, session_id
        ORDER BY timestamp
    )
),
    MAX_TIMESTAMP
AS
(
    SELECT
        user_id
        , session_id
        , MAX(timestamp) AS timestamp
    FROM df
    GROUP BY user_id, session_id
),
    WINDOWED_ROUTES
AS
(
    SELECT
        user_id
        , session_id
        , timestamp
        , COLLECT_LIST(event_page) OVER window_spec AS route
    FROM EXTRACT_ERRORS
    WHERE
        cumsum_error = 0
        AND event_type = 'page'
    WINDOW window_spec AS (
        PARTITION BY user_id, session_id
        ORDER BY timestamp
    )
)
SELECT
    CONCAT_WS('-', W.route) AS route
    , COUNT(*) AS count
FROM
    WINDOWED_ROUTES W
JOIN
    MAX_TIMESTAMP M
ON
    W.user_id = M.user_id
    AND W.session_id = M.session_id
    AND W.timestamp = M.timestamp
GROUP BY
    W.route
ORDER BY
    count DESC
LIMIT 30

"""
).show(30)

+--------------------+-----+
|               route|count|
+--------------------+-----+
|                main| 2392|
|        main-archive|  337|
|         main-rabota|  315|
|       main-internet|  265|
|          main-bonus|  247|
|           main-news|  234|
|         main-online|  179|
|        main-tariffs|  178|
|          main-vklad|  165|
| main-archive-rabota|   53|
|main-internet-arc...|   50|
| main-rabota-archive|   43|
|main-archive-inte...|   42|
|   main-archive-news|   42|
|main-internet-rabota|   41|
|  main-bonus-archive|   40|
|   main-rabota-bonus|   39|
|  main-archive-bonus|   38|
|   main-bonus-rabota|   38|
|main-rabota-internet|   37|
|main-archive-tariffs|   35|
|    main-news-rabota|   34|
|     main-news-bonus|   33|
|main-tariffs-inte...|   32|
|  main-internet-news|   32|
|main-tariffs-archive|   32|
|        main-digital|   31|
| main-internet-bonus|   31|
|    main-rabota-news|   29|
| main-archive-online|   28|
+--------------------+-----+



## 3 Using Spark DataFrame API

In [205]:
window_spec = Window.partitionBy(*["user_id", "session_id"]).orderBy("timestamp")

(
    df
    .withColumn("error_flag", F.when(F.col("event_type").contains("error"), 1).otherwise(0))
    .withColumn("cumsum_error", F.sum("error_flag").over(window_spec))
    .filter(F.col("cumsum_error") == 0)
    .filter(F.col("event_type")=="page")
    .withColumn("route", F.collect_list("event_page").over(window_spec))
    .join(
        F.broadcast(
            df
            .groupBy(*["user_id", "session_id"])
            .agg(F.max(F.col("timestamp")).alias("timestamp"))
        ),
        on=["user_id", "session_id", "timestamp"],
        how="inner"
    )
    .withColumn("route", F.concat_ws("-", "route"))
    .groupBy("route")
    .count()
    .orderBy(F.col("count").desc())
    .limit(30)
    .show(30)
)

+--------------------+-----+
|               route|count|
+--------------------+-----+
|                main| 2392|
|        main-archive|  337|
|         main-rabota|  315|
|       main-internet|  265|
|          main-bonus|  247|
|           main-news|  234|
|         main-online|  179|
|        main-tariffs|  178|
|          main-vklad|  165|
| main-archive-rabota|   53|
|main-internet-arc...|   50|
| main-rabota-archive|   43|
|   main-archive-news|   42|
|main-archive-inte...|   42|
|main-internet-rabota|   41|
|  main-bonus-archive|   40|
|   main-rabota-bonus|   39|
|  main-archive-bonus|   38|
|   main-bonus-rabota|   38|
|main-rabota-internet|   37|
|main-archive-tariffs|   35|
|    main-news-rabota|   34|
|     main-news-bonus|   33|
|main-tariffs-archive|   32|
|  main-internet-news|   32|
|main-tariffs-inte...|   32|
|        main-digital|   31|
| main-internet-bonus|   31|
|    main-rabota-news|   29|
| main-archive-online|   28|
+--------------------+-----+



In [91]:
rdd_top_routes = rdd_routes_count.takeOrdered(20, key=lambda x: -x[1])

2023-10-05 19:21:27,934 WARN scheduler.TaskSetManager: Lost task 1.0 in stage 73.0 (TID 664) (475c97847897 executor 2): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/tmp/hadoop-jovyan/nm-local-dir/usercache/jovyan/appcache/application_1696475076700_0002/container_1696475076700_0002_01_000003/pyspark.zip/pyspark/worker.py", line 619, in main
    process()
  File "/tmp/hadoop-jovyan/nm-local-dir/usercache/jovyan/appcache/application_1696475076700_0002/container_1696475076700_0002_01_000003/pyspark.zip/pyspark/worker.py", line 609, in process
    out_iter = func(split_index, iterator)
  File "/usr/local/spark/python/pyspark/rdd.py", line 3122, in pipeline_func
  File "/usr/local/spark/python/pyspark/rdd.py", line 3122, in pipeline_func
  File "/usr/local/spark/python/pyspark/rdd.py", line 540, in func
    >>> rdd = sc.parallelize(range(500), 1)
  File "/usr/local/spark/python/pyspark/rdd.py", line 2147, in combineLocally
    return merger.items()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 73.0 failed 4 times, most recent failure: Lost task 1.3 in stage 73.0 (TID 672) (475c97847897 executor 2): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/tmp/hadoop-jovyan/nm-local-dir/usercache/jovyan/appcache/application_1696475076700_0002/container_1696475076700_0002_01_000003/pyspark.zip/pyspark/worker.py", line 619, in main
    process()
  File "/tmp/hadoop-jovyan/nm-local-dir/usercache/jovyan/appcache/application_1696475076700_0002/container_1696475076700_0002_01_000003/pyspark.zip/pyspark/worker.py", line 609, in process
    out_iter = func(split_index, iterator)
  File "/usr/local/spark/python/pyspark/rdd.py", line -1, in pipeline_func
  File "/usr/local/spark/python/pyspark/rdd.py", line -1, in pipeline_func
  File "/usr/local/spark/python/pyspark/rd

In [138]:
def pretty_print(top_routes):
    print("+--------------------+-----+")
    print("|               route|count|")
    print("+--------------------+-----+")
    
    for route, count in top_routes:
        truncated_route = (route[:17] + '...') if len(route) > 20 else route.ljust(20, ' ')
        aligned_count = str(count).rjust(5, ' ')
        print(f"|{truncated_route}|{aligned_count}|")
    
    print("+--------------------+-----+")

    
pretty_print(rdd_top_routes)

+--------------------+-----+
|               route|count|
+--------------------+-----+
|tariffs-main-onli...|    1|
|rabota-main-archi...|    1|
|archive-rabota-ar...|    1|
|news-online-bonus...|    1|
|news-main-news-ra...|    1|
|online-news-main-...|    1|
|news-archive-vkla...|    1|
|rabota-main-bonus...|    1|
|bonus-archive-mai...|    1|
|internet-archive-...|    1|
|internet-rabota-m...|    1|
|archive-online-ta...|    1|
|bonus-news-rabota...|    1|
|archive-internet-...|    1|
|news-vklad-news-m...|    1|
|bonus-archive-vkl...|    1|
|archive-main-arch...|    1|
|tariffs-bonus-vkl...|    1|
|rabota-archive-di...|    1|
|bonus-rabota-main...|    1|
+--------------------+-----+
